In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import random

In [2]:
!kaggle competitions download -c titanic

titanic.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
import zipfile

zp = zipfile.ZipFile('titanic.zip') 
df = pd.read_csv(zp.open('train.csv'))

In [4]:
df = df.drop(['PassengerId', 'Name', 'Parch', 'Ticket', 'Cabin'], axis = 1)

df.loc[df['SibSp'] >= 1, 'SibSp'] = 1
df['Embarked'] = df['Embarked'].fillna('S')

df = df.dropna(subset = ['Age'])
df['Age'] = df['Age'].astype(int)

df.head()

,Survived,Pclass,Sex,Age,SibSp,Fare,Embarked
0,0,3,male,22,1,7.2500,S
1,1,1,female,38,1,71.2833,C
2,1,3,female,26,0,7.9250,S
3,1,1,female,35,1,53.1000,S
4,0,3,male,35,0,8.0500,S


In [5]:
df = pd.concat([df, pd.get_dummies(df['Sex'], prefix='Sex')],axis=1)
df = pd.concat([df, pd.get_dummies(df['Pclass'], prefix='Pclass')],axis=1)
df = pd.concat([df, pd.get_dummies(df['Embarked'], prefix='Embarked')],axis=1)
  
df = df.drop(['Sex', 'Pclass', 'Embarked'], axis = 1)
df.head()

,Survived,Age,SibSp,Fare,Sex_female,Sex_male,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S
0,0,22,1,7.2500,0,1,0,0,1,0,0,1
1,1,38,1,71.2833,1,0,1,0,0,1,0,0
2,1,26,0,7.9250,1,0,0,0,1,0,0,1
3,1,35,1,53.1000,1,0,1,0,0,0,0,1
4,0,35,0,8.0500,0,1,0,0,1,0,0,1


In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.drop(['Survived'], axis = 1), df['Survived'], test_size=0.2)
y_train

371    0
591    1
873    0
352    0
430    1
      ..
132    0
553    1
465    0
366    1
575    0
Name: Survived, Length: 571, dtype: int64

In [7]:
class NN:
    def __init__(self, n_hidden_units = 100, learning_rate = 0.001):
        self.learning_rate = learning_rate
        self.n_hidden_units = n_hidden_units
        
        self.activation_func = lambda x: 1 / (1 + np.exp(-x))
        self.activation_func_derivative = lambda x: np.exp(-x) / (1 + np.exp(-x))**2
            
            
    def fit(self, X_train, y_train):
        self.X = X_train.to_numpy()
        self.y = y_train.to_numpy().reshape((-1, 1))
        
        #z1 = x * W1 + b1 (col * len x len * hd) + col * cl
        #a1 = act_func(z1) 
        #z2 = a1 * W2 + b2
        #a2 = act_func(z2)
        
        self.W1 = np.random.normal(0, 0.01, (self.X.shape[1], self.n_hidden_units))
        self.b1 = np.ones((1, self.n_hidden_units))
        self.W2 = np.random.normal(0, 0.01, (self.n_hidden_units, 1))
        self.b2 = np.ones((1, 1))
        
        for i in range(100):
            #Forward propagation
            z1 = self.X.dot(self.W1) + self.b1
            a1 = self.activation_func(z1)
            z2 = a1.dot(self.W2) + self.b2
            a2 = self.activation_func(z2)
            
            #Backpropagation
            error = a2 - self.y
            #print(np.absolute(error).sum())
            
            d2 = error * self.activation_func_derivative(a2)
            d1 = d2.dot(self.W2.T) * self.activation_func_derivative(a1)
            
            dW2 = a1.T.dot(d2)
            dW1 = self.X.T.dot(d1)
            db2 = d2.sum()
            db1 = d1.sum()
            
            self.W1 -= self.learning_rate * dW1
            self.W2 -= self.learning_rate * dW2
            self.b1 -= self.learning_rate * db1
            self.b2 -= self.learning_rate * db2
            
    def predict_proba(self, X_test):
        return self.activation_func(self.activation_func(X_test.dot(self.W1) + self.b1).dot(self.W2) + self.b2)
    
    def predict(self, X_test):
        y = self.predict_proba(X_test)
        
        y[y > 0.5] = 1
        y[y <= 0.5] = 0
           
        return y

In [8]:
clf = NN()
print('sigmoid', clf.activation_func(np.array([0.2, 0.5])))
print('sigmoid derivative', clf.activation_func_derivative(np.array([0.2, 0.5])))

sigmoid [0.549834   0.62245933]
sigmoid derivative [0.24751657 0.23500371]


In [9]:
clf.fit(X_train, y_train)

from sklearn.metrics import accuracy_score
accuracy_score(y_test, clf.predict(X_test))

0.7342657342657343

In [10]:
from sklearn.neural_network import MLPClassifier

clf2 = MLPClassifier(activation = 'logistic', solver = 'sgd')
clf2.fit(X_train, y_train)

accuracy_score(y_test, clf2.predict(X_test))

0.7412587412587412

In [11]:
clf2 = MLPClassifier()
clf2.fit(X_train, y_train)

accuracy_score(y_test, clf2.predict(X_test))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


0.8111888111888111

[Link to code explanation](https://www.analyticsvidhya.com/blog/2020/07/neural-networks-from-scratch-in-python-and-r/)